# ❄️  [Day 7](https://adventofcode.com/2019/day/7)

In [46]:
def run_program(p, inputs, init_op=0, early_exit=False):
    #Inputs are given in reverse order (pop)
    op = init_op
    last_diagnostic = None
    while p[op] != 99:
        codes = "%05d" % p[op]
        codes = [int(codes[0]), int(codes[1]), int(codes[2]), int(codes[3:])]
        assert codes[0] == 0
        # Outputs
        if codes[-1] == 4:
            last_diagnostic = p[p[op + 1]]
            op += 2
            if early_exit:
                return last_diagnostic, 1, op
        # Inputs
        elif codes[-1] == 3:
            assert codes[1] == codes[2] == 0
            p[p[op + 1]] = inputs.pop()
            op += 2
        else:
            x, y = p[op + 1:op + 3]
            x = p[x] if codes[2] == 0 else x
            y = p[y] if codes[1] == 0 else y
            # addition and multiplication
            if codes[-1] in [1, 2]:
                p[p[op + 3]] = x + y if codes[-1] == 1 else x * y
                op += 4
            # Comparison result
            elif codes[-1] == 7:
                p[p[op + 3]] = int(x < y)
                op += 4
            elif codes[-1] == 8:
                p[p[op + 3]] = int(x == y)
                op += 4
            # Jump if eq
            elif (codes[-1] == 5 and x != 0) or (codes[-1] == 6 and x == 0):
                op = y  
            # Jump instruction that failed their test
            else:
                op += 3
    return last_diagnostic, 0, op

def max_signal(program, num_amplifiers=5, taken=[], in_=0):
    current_max = 0
    assignment = []
    for s in [x for x in range(num_amplifiers) if x not in taken]:
        output, _, _ = run_program([x for x in program], [in_, s], early_exit=False)
        if len(taken) == num_amplifiers - 1:
            return output, taken + [s]
        else:
            c, a = max_signal(program, in_=output, taken=taken + [s])
        if c > current_max:
            current_max = c
            assignment = a
    return current_max, assignment

In [47]:
with open("inputs/day7.txt", 'r') as f:
    inputs = list(map(int, f.read().split(',')))
    
print("Maximum thruster signal {} with phase settings {}".format(*max_signal(inputs)))

Maximum thruster signal 422858 with phase settings [3, 1, 4, 2, 0]


In [48]:
import itertools

def max_signal_recursive(program, num_amplifiers=5, taken=[], in_=0):
    current_max = 0
    best_assignment = []
    current_thrust_value = 0
    program_states = [[x for x in program] for _ in range(num_amplifiers)]
    op_states = [0 for _ in range(num_amplifiers)]
    for assignment in itertools.permutations([5, 6, 7, 8, 9]):
        # first iteration
        output, _, op_states[0] = run_program(
            program_states[0], [in_, assignment[0]], early_exit=True)
        for i in range(1, 5):
            output, _, op_states[i] = run_program(
                program_states[i], [output, assignment[i]], early_exit=True)
        current_thrust_value = output
        # feedback loop
        amplifier_index = 0
        while 1:
            output, _, op_states[amplifier_index] = run_program(
                program_states[amplifier_index], [output], early_exit=True,
                init_op=op_states[amplifier_index])
            amplifier_index += 1
            amplifier_index %= 5
            if not amplifier_index:
                if output is None:
                    if current_thrust_value > current_max:
                        current_max = current_thrust_value
                        best_assignment = assignment
                    break
                else:
                    current_thrust_value = output
    return current_max, best_assignment

In [49]:
print("Maximum thruster signal {} with phase settings {}".format(*max_signal_recursive(inputs)))

Maximum thruster signal 14897241 with phase settings (7, 8, 9, 6, 5)
